# Interactive Voltage and Current Display

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import tresonator as T
import skrf as rf

In [3]:
# matched resonator measurement for SSA84
tres_exp = rf.Network('data/SSA84_TaskB_resonator_matched_3.s1p')
tres_exp.frequency.unit = 'MHz'

In [38]:
# SSA84 optimized parameters to match the measurement above (done in another notebook)
P_in=120 # kW
L_DUT_opt=0.023  # m
L_CEA_opt=0.1473  # m
Z_DUT_opt=4.7e-06  # Ohm
Z_CEA_opt=0.023  # Ohm
add_loss_opt=0.9  # Multiplicative coefficient to total conductivity

In [46]:
@interact(freq=(60,65,0.01), P_in=(10, 200,1), 
         L_DUT=(1, 200, 0.1), L_CEA=(1, 200, 0.1), 
         Z_DUT=(1e-6, 0.1, 1e-6), Z_CEA=(1e-6, 0.1, 1e-6))
def plot_voltage_current_vs_frequency(freq=61.71, P_in=P_in, 
                                      L_DUT=L_DUT_opt*1e3, L_CEA=L_CEA_opt*1e3, 
                                      Z_DUT=Z_DUT_opt, Z_CEA=Z_CEA_opt, 
                                      additional_losses=add_loss_opt):
    cfg_opt = T.Configuration(freq*1e6, P_in*1e3, L_DUT*1e-3, L_CEA*1e-3, 
                               Z_short_DUT=Z_DUT, Z_short_CEA=Z_CEA, 
                               additional_losses=additional_losses)

    # Calculates the voltage and current along the transmission lines
    L_CEA, L_DUT, V_CEA, V_DUT, I_CEA, I_DUT = cfg_opt.voltage_current()

    # Plotting V,I
    fig, ax = plt.subplots(3,1, sharex=False, figsize=(10,5))
    ax[0].plot(L_CEA, np.abs(V_CEA)/1e3,-L_DUT, np.abs(V_DUT)/1e3,  lw=2)
    ax[0].set_ylim(0, 60)
    ax[0].grid(True)
    ax[0].set_xlim(min(-L_DUT), max(L_CEA))
    ax[0].axvline(x=cfg_opt.L_Vprobe_CEA_fromT, ls='--', color='gray', lw=3)
    ax[0].axvline(x=-cfg_opt.L_Vprobe_DUT_fromT, ls='--', color='gray', lw=3)
    ax[0].set_ylabel('|V| [kV]', fontsize=14)
    ax[0].tick_params(labelsize=14)

    ax[1].plot(L_CEA, np.abs(I_CEA),-L_DUT, np.abs(I_DUT),  lw=2)
    ax[1].set_ylim(0, 3000)
    ax[1].grid(True)
    ax[1].axvline(x=cfg_opt.L_Vprobe_CEA_fromT, ls='--', color='gray', lw=3)
    ax[1].axvline(x=-cfg_opt.L_Vprobe_DUT_fromT, ls='--', color='gray', lw=3)
    ax[1].set_xlabel('L [m]', fontsize=14)
    ax[1].set_ylabel('|I| [A]', fontsize=14)
    ax[1].tick_params(labelsize=14)
    ax[1].set_xlim(min(-L_DUT), max(L_CEA))
    
    cfg_opt.circuit(rf.Frequency(61, 62, 1001, unit='MHz')).network.plot_s_db(ax=ax[2])
    tres_exp.plot_s_db(ax=ax[2], color='darkgray', ls='--')
    ax[2].set_xlim(61e6, 62e6)
    ax[2].set_ylim(-50, 0)

    


interactive(children=(FloatSlider(value=61.71, description='freq', max=65.0, min=60.0, step=0.01), IntSlider(v…

# Deducing the current at the short from the voltage at the probes

In [24]:
freq = 61.70
cfg_opt = T.Configuration(freq*1e6, P_in, L_DUT_opt, L_CEA_opt, 
                            Z_short_DUT=Z_DUT_opt, Z_short_CEA=Z_CEA_opt, 
                               additional_losses=add_loss_opt)

# varying the probe position to determine the sensibility 
cfg_opt.L_Vprobe_CEA_fromT = 1.236  # +/- 3 mm
cfg_opt.L_Vprobe_DUT_fromT = 0.660  # +/- 3 mm?

L_CEA, L_DUT, V_CEA, V_DUT, I_CEA, I_DUT = cfg_opt.voltage_current()

I_max_CEA = np.abs(I_CEA[-1])
I_max_DUT = np.abs(I_DUT[-1])
V_max_CEA = np.max(np.abs(V_CEA))
V_max_DUT = np.max(np.abs(V_DUT))
V_probeCEA = np.abs(V_CEA[np.argmin(np.abs(L_CEA - cfg_opt.L_Vprobe_CEA_fromT))])
V_probeDUT = np.abs(V_DUT[np.argmin(np.abs(L_DUT - cfg_opt.L_Vprobe_DUT_fromT))])

In [25]:
coeffVprobeCEA_2_Vmax = V_max_CEA/V_probeCEA
print('VprobeCEA -> V max CEA:', coeffVprobeCEA_2_Vmax)

coeffVprobeDUT_2_Vmax = V_max_DUT/V_probeDUT
print('VprobeDUT -> V max DUT:',coeffVprobeDUT_2_Vmax)

coeffVprobeCEA_2_Imax = I_max_CEA/V_probeCEA
print('VprobeCEA -> I short CEA:', coeffVprobeCEA_2_Imax)

coeffVprobeDUT_2_Imax = I_max_DUT/V_probeDUT
print('VprobeDUT -> I short DUT:',coeffVprobeDUT_2_Imax)

VprobeCEA -> V max CEA: 1.0004090849277916
VprobeDUT -> V max DUT: 1.2037921987423104
VprobeCEA -> I short CEA: 0.033727575551000435
VprobeDUT -> I short DUT: 0.0637854526843531


In [ ]:
# Plotting V,I
fig, ax = plt.subplots(3,1, sharex=False, figsize=(10,5))

ax[0].plot(L_CEA, np.abs(V_CEA)/1e3,-L_DUT, np.abs(V_DUT)/1e3,  lw=2)
ax[0].set_ylim(0, 60)
ax[0].grid(True)
ax[0].set_xlim(min(-L_DUT), max(L_CEA))
ax[0].axvline(x=cfg_opt.L_Vprobe_CEA_fromT, ls='--', color='gray', lw=3)
ax[0].axvline(x=-cfg_opt.L_Vprobe_DUT_fromT, ls='--', color='gray', lw=3)
ax[0].set_ylabel('|V| [kV]', fontsize=14)
ax[0].tick_params(labelsize=14)

ax[1].plot(L_CEA, np.abs(I_CEA),-L_DUT, np.abs(I_DUT),  lw=2)
ax[1].set_ylim(0, 3000)
ax[1].grid(True)
ax[1].axvline(x=cfg_opt.L_Vprobe_CEA_fromT, ls='--', color='gray', lw=3)
ax[1].axvline(x=-cfg_opt.L_Vprobe_DUT_fromT, ls='--', color='gray', lw=3)
ax[1].set_xlabel('L [m]', fontsize=14)
ax[1].set_ylabel('|I| [A]', fontsize=14)
ax[1].tick_params(labelsize=14)
ax[1].set_xlim(min(-L_DUT), max(L_CEA))

cfg_opt.circuit(rf.Frequency(61, 62, 1001, unit='MHz')).network.plot_s_db(ax=ax[2])
ax[2].set_xlim(61e6, 62e6)